In [ ]:
# only for taurus
import os
print(os.getcwd())
os.chdir("/home/jori152b/DIR/horse/jori152b-medinf/KP_MedInf/model_development")
# print(os.getcwd())

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
df = pd.read_csv('../data/analysis/data_preprocessed_extended_before_resampling.csv', parse_dates=['charttime'])

In [ ]:
x1 = pd.read_csv('../data/preprocessed/preprocessed_data_extended_6H.csv', parse_dates=['charttime'])
x2 = pd.read_csv('../data/preprocessed/preprocessed_data.csv', parse_dates=['charttime'])

In [ ]:
print(x1.columns)
print(x2.columns)

In [ ]:
# get the difference in columns between x1 and x2
print(x1.columns.difference(x2.columns))

In [ ]:
stats = df.describe()
print(stats)    
# save to csv
stats.to_csv('data/analysis/data_preprocessed_extended_stats.csv', index=False)

In [ ]:
# Initialize a dictionary to store results
results = {}

# List of columns to analyze (excluding the datetime and other non-measurement columns)
columns_to_analyze = df.columns.difference(['icustay_id', 'charttime', 'hadm_id', 'subject_id'])

# Group by icustay_id
grouped = df.groupby('icustay_id')

# Iterate through each column to analyze
for column in columns_to_analyze:
    frequencies = []
    mean_time_diffs = []
    std_time_diffs = []
    max_time_diffs = []
    min_time_diffs = []

    # Iterate through each group (each icustay_id)
    for icustay_id, group in grouped:
        valid_rows = group[group[column].notna()]

        frequency = valid_rows.shape[0]
        if frequency > 1:
            time_diffs = valid_rows['charttime'].diff().dropna().dt.total_seconds() / 60  # in minutes

            mean_time_diff = time_diffs.mean()
            std_time_diff = time_diffs.std()
            max_time_diff = time_diffs.max()
            min_time_diff = time_diffs.min()
        else:
            mean_time_diff = std_time_diff = max_time_diff = min_time_diff = None

        frequencies.append(frequency)
        mean_time_diffs.append(mean_time_diff)
        std_time_diffs.append(std_time_diff)
        max_time_diffs.append(max_time_diff)
        min_time_diffs.append(min_time_diff)

    # Aggregate the statistics across all icustay_id groups
    overall_frequency = sum(frequencies)
    overall_mean_time_diff = pd.Series(mean_time_diffs).mean()
    overall_std_time_diff = pd.Series(std_time_diffs).mean()
    overall_max_time_diff = pd.Series(max_time_diffs).max()
    overall_min_time_diff = pd.Series(min_time_diffs).min()

    results[column] = {
        'frequency': overall_frequency,
        'mean_time_diff': overall_mean_time_diff,
        'std_time_diff': overall_std_time_diff,
        'max_time_diff': overall_max_time_diff,
        'min_time_diff': overall_min_time_diff
    }

# Convert results to a DataFrame for better visualization
results_df = pd.DataFrame(results).T
print(results_df)

# Optionally, save the results to a CSV file
results_df.to_csv('../data/analysis/measurement_statistics_by_icustay.csv')


In [ ]:
# drop all the columns in string format
X.drop(['first_hosp_stay'], axis=1, inplace = True)
X.drop(['first_icu_stay'], axis=1, inplace = True)
X.drop(['ethnicity'], axis=1, inplace = True)
X.drop(['admittime'], axis=1, inplace = True)
X.drop(['dischtime'], axis=1, inplace = True)
X.drop(['intime'], axis=1, inplace = True)
X.drop(['outtime'], axis=1, inplace = True)
X.drop(['dod'], axis=1, inplace = True)
X.drop(['charttime'], axis=1, inplace = True)


In [ ]:
X = x1

In [ ]:
# Calculate correlations
correlations = X.corr()['aki_stage'].drop('aki_stage').sort_values(ascending=False)

# Create bar chart
plt.figure(figsize=(20,10))
correlations.plot(kind='bar')
plt.title('Correlation of all features with the target variable')
plt.ylabel('Correlation')
plt.show()

In [ ]:
# calculate the frequency of each attribute, i.e. the distribution of non nan values
X.apply(lambda x: x.count(), axis=0)

In [ ]:
# read np dict
results_lstm = np.load('model_development/notebooks/data/results_LSTM.npy', allow_pickle=True).item()   
results_xgb = np.load('model_development/notebooks/data/results.npy', allow_pickle=True).item()

In [ ]:
print(results_lstm)
print(results_xgb)

In [ ]:
# Extract dataset names and metrics
datasets = list(results_lstm.keys())
metrics = ['test_accuracy', 'test_roc_auc', 'test_pr_auc']

# Set up the plot
fig, axs = plt.subplots(3, 1, figsize=(12, 18))
fig.suptitle('Comparison of LSTM and XGBoost Models', fontsize=16)

# Plot each metric
for i, metric in enumerate(metrics):
    lstm_values = [results_lstm[dataset][metric] for dataset in datasets]
    xgb_values = [results_xgb[dataset][metric] for dataset in datasets]
    
    x = np.arange(len(datasets))
    width = 0.35
    
    axs[i].bar(x - width/2, lstm_values, width, label='LSTM')
    axs[i].bar(x + width/2, xgb_values, width, label='XGBoost')
    
    axs[i].set_ylabel(metric)
    axs[i].set_title(f'{metric.capitalize()} Comparison')
    axs[i].set_xticks(x)
    axs[i].set_xticklabels(datasets, rotation=45, ha='right')
    axs[i].legend()

    # Add value labels on top of each bar
    for j, v in enumerate(lstm_values):
        axs[i].text(j - width/2, v, f'{v:.3f}', ha='center', va='bottom')
    for j, v in enumerate(xgb_values):
        axs[i].text(j + width/2, v, f'{v:.3f}', ha='center', va='bottom')

plt.tight_layout()
plt.show()

In [ ]:
feature_selection_results = np.load('data/optimal_features.npy', allow_pickle=True)

In [ ]:
feature_selection_results = np.load('data/feature_importances.npy', allow_pickle=True)


In [ ]:
print(feature_selection_results)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

results = feature_selection_results
# Extract data from the results
n_features = [result['n_features'] for result in results]
val_roc_auc = [result['val_roc_auc'] for result in results]
val_pr_auc = [result['val_pr_auc'] for result in results]

# Create the plot
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 16))

# Plot ROC AUC and PR AUC
ax1.plot(n_features, val_roc_auc, 'b-o', label='ROC AUC')
ax1.plot(n_features, val_pr_auc, 'r-o', label='PR AUC')
ax1.set_xlabel('Number of Features')
ax1.set_ylabel('AUC Score')
ax1.set_title('ROC AUC and PR AUC vs Number of Features')
ax1.legend()
ax1.grid(True)

# Find the best performing model (highest ROC AUC)
best_model_index = np.argmax(val_roc_auc)
best_model = results[best_model_index]

# Get top 15 features from the best model
top_features = best_model['sorted_importance']
feature_names = [feature[0] for feature in top_features]
feature_importance = [feature[1] for feature in top_features]

# Plot feature importance for the best model
ax2.barh(range(len(feature_names)), feature_importance, align='center')
ax2.set_yticks(range(len(feature_names)))
ax2.set_yticklabels(feature_names)
ax2.invert_yaxis()  # Labels read top-to-bottom
ax2.set_xlabel('Feature Importance')
ax2.set_title(f'Top Features (Best model with {best_model["n_features"]} features)')

plt.tight_layout()
plt.show()